In [1]:
import requests
import pandas as pd
import os

In [2]:
download_dir = "./downloads"
os.makedirs(download_dir, exist_ok=True)

In [3]:
# 3. 災害タイプで「Earthquake」のみ抽出
events = requests.get("http://localhost:8000/get_events").json()
target_events = [e for e in events if e['disaster_type'].lower() == 'earthquake']
target_events

[{'name': 'Earthquake in Central Tajikistan',
  'description': '2025-04-13: Earthquake in Central Tajikistan on 13 April, 2025',
  'disaster_type': 'Earthquake',
  'country': 'Tajikistan',
  'country_iso3': 'TJK',
  'occurrence_date': '2025-04-13',
  'sa_activation_date': '2025-04-13',
  'requester': 'Central Asian Institute of Applied Geosciences (CAIAG)',
  'escalation_to_charter': 'No',
  'glide_number': 'EQ-2025-000049-TJK',
  'additional_metadata': None,
  'files': [],
  'url': 'https://sentinel-asia.org/EO/2025/article20250413TJ.html'},
 {'name': 'Earthquake in Mandalay, Myanmar',
  'description': '2025-03-28: Earthquake in Mandalay, Myanmar on 28 March, 2025',
  'disaster_type': 'Earthquake',
  'country': 'Myanmar',
  'country_iso3': 'MMR',
  'occurrence_date': '2025-03-28',
  'sa_activation_date': '2025-03-28',
  'requester': 'Myanmar Information Management Unit (MIMU)',
  'escalation_to_charter': 'No',
  'glide_number': 'EQ-2025-000043-MMR',
  'additional_metadata': None,
  'f

In [4]:
product_list = []
for event in target_events:
    if not event.get('url'):
        continue
    products_url = "http://localhost:8000/get_products"
    res = requests.get(products_url, params={"url": event['url']})
    if res.status_code != 200:
        continue
    products = res.json()
    kmz_products = [p for p in products if p.get('file_type') == 'kmz']
    for prod in kmz_products:
        product_list.append({
            "event_name": event['name'],
            "product_title": prod.get('title'),
            "download_url": prod.get('download_url')
        })
df_products = pd.DataFrame(product_list)

In [5]:
first_row = df_products.iloc[0]
url = first_row['download_url']
filename = url.split('/')[-1]
save_path = os.path.join(download_dir, filename)

In [6]:
try:
    print(f"Downloading: {url}")
    r = requests.get(url)
    r.raise_for_status()
    with open(save_path, 'wb') as f:
        f.write(r.content)
    print(f"Saved: {save_path}")
except Exception as e:
    print(f"Failed to download {url}: {e}")

Downloading: https://sentinel-asia.org/EO/2025/article20250413TJ/EOS-RS_20250416_DPM_S1/EOS-RS_20250416_DPM_S1_Tajikistan_Afghanistan_Earthquakes_v0.7.kmz
Saved: ./downloads/EOS-RS_20250416_DPM_S1_Tajikistan_Afghanistan_Earthquakes_v0.7.kmz


In [7]:
# ファイルが本当に保存されたか確認
if os.path.exists(save_path):
    print(f"ファイルが正常に保存されました: {save_path}")
else:
    print("ファイルが保存されていません。")

ファイルが正常に保存されました: ./downloads/EOS-RS_20250416_DPM_S1_Tajikistan_Afghanistan_Earthquakes_v0.7.kmz
